### 0) Description

This notebook contains code to pull seasonal data (2000-2020) from the following https://www.sports-reference.com/ websites:
    - College Football: https://www.sports-reference.com/cfb/
    - NFL: https://www.pro-football-reference.com/
    
Tables/sites to be scripted:
    - NCAA team Ratings: https://www.sports-reference.com/cfb/years/yyyy-ratings.html
    - NCAA Team Offense: https://www.sports-reference.com/cfb/years/yyyy-team-offense.html
    - NCAA Team Defense: https://www.sports-reference.com/cfb/years/yyyy-team-defense.html
    - NCAA Player Passing Stats: https://www.sports-reference.com/cfb/years/yyyy-passing.html
    - NCAA Player Receiving Stats: https://www.sports-reference.com/cfb/years/yyyy-receiving.html
    - NCAA Player Rushing Stats: https://www.sports-reference.com/cfb/years/yyyy-rushing.html
    - NCAA Player Kicking Stats: https://www.sports-reference.com/cfb/years/yyyy-kicking.html
    - NCAA Player Punting Stats: https://www.sports-reference.com/cfb/years/yyyy-punting.html
    - NCAA Season Summary: https://www.sports-reference.com/cfb/years/yyyy.html
    - NFL Combine Results: https://www.pro-football-reference.com/draft/yyyy-combine.htm
    - NFL Draft Results: https://www.pro-football-reference.com/years/yyyy/draft.htm
    - NFL Standings and Team Stats: https://www.pro-football-reference.com/years/yyyy/
    

### 1) Import packages

In [5]:
import requests                   # to make html requests
from bs4 import BeautifulSoup     # to pull data from html websites
import pandas as pd               # to create dataframes and concat dataframes and read html tables
import time                       # to set delays between requests
import numpy as np                # to choose random elements from list
import re

### 2) Set parameters

In [140]:
# 2000-2020 - e.g 2000 nfl draft is based on 1999 college season
years_nfl = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019', '2020']

# 1999-2019 - e.g 1999 season ends in 2000 and players get drafted in 2000
years_ncaa = ['1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']

# list of tables for player stats
player_tables = ['passing','rushing','receiving','kicking','punting']

# list of delays to put between get requests
delays = [5, 6, 7, 8, 9, 10, 14, 17, 19, 23, 27, 29, 31, 33, 35, 42, 45]

# set request headers based on website
headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'http://www.google.com/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
}

### 3) Define functions to pull data

#### 3.1 Pull combine results

In [9]:
# get combine data
def pullCombineData(years):
    # a list to store dataframes -- 1 df per year
    dfs = []
    # a list to store any errors that may come up while scraping
    error_list = []
    
    # iterate over years
    for year in years:
        # use try/except block to catch and inspect any urls that cause an error
        try:
            # set url
            url = 'https://www.pro-football-reference.com/draft/' + year + '-combine.htm'
            
            # put random delays between get requests
            delay = np.random.choice(delays)
            time.sleep(delay)
            # make get request with headers
            html = requests.get(url, headers=headers)

            # create the BeautifulSoup object
            soup = BeautifulSoup(html.content, "lxml")
            # get combine table
            table = soup.find('table', {'id': 'combine'})
            # get column headers
            column_headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
            
            # read table as dataframe
            df = pd.read_html(str(table))[0]
            # remove duplicate header rows
            df.drop_duplicates(keep=False, inplace=True) 

            # get player ids and ncaa links, if not exist put N/A
            player_ids = []
            ncaa_links = []
            for tbody in table.find_all('tbody'):
                for tr in tbody.find_all('tr'):
                    for th in tr.find_all('th'):
                        if th.text not in column_headers:
                            try:
                                player_ids.append(th['data-append-csv'])
                            except KeyError:
                                player_ids.append('N/A')
                    for td in tr.find_all('td'):
                            if td['data-stat'] == "college":
                                if td.find_all('a'):
                                    for a in td.find_all('a'):
                                        ncaa_links.append(a['href'])
                                else:
                                    ncaa_links.append('N/A')
            
            # insert year, player id, ncaa link to table
            df.insert(0, "NCAA_Link", ncaa_links)
            df.insert(0, "Player_ID", player_ids)
            df.insert(0, "Year", year)
            
            # append df to dfs list
            dfs.append(df)

        except Exception as e:
            # store the url and the error it causes in a list
            error = [url, e] 
            # append it to the list of errors
            error_list.append(error)
    
    # print errors
    print(error_list)
    #  in one DataFrame
    combines_df = pd.concat(dfs, ignore_index=True)
    combines_df.to_csv('nfl_combine_2000_2020.csv', index=False)

In [10]:
pullCombineData(years_nfl)

[]


#### 3.2 Pull draft results

In [2]:
# get draft data
def pullDraftData(years):
    # a list to store dataframes -- 1 df per year
    dfs = []
    # a list to store any errors that may come up while scraping
    error_list = []
    
    # iterate over years
    for year in years:
        # use try/except block to catch and inspect any urls that cause an error
        try:
            # set url
            url = 'https://www.pro-football-reference.com/years/' + year + '/draft.htm'
            
            # put random delays between get requests
            delay = np.random.choice(delays)
            time.sleep(delay)
            # make get request with header
            html = requests.get(url, headers=headers)

            # create the BeautifulSoup object
            soup = BeautifulSoup(html.content, "lxml")
            # get draft table
            table = soup.find('table', {'id': 'drafts'})
            
            # read table as dataframe
            df = pd.read_html(str(table))[0]
            # remove duplicate header rows
            df.drop_duplicates(keep=False, inplace=True)
            # remove multiheaders (first header row)
            df.columns = df.columns.droplevel(0)

            player_ids = []
            ncaa_links = []
            for tbody in table.find_all('tbody'):
                for tr in tbody.find_all('tr'):
                    for td in tr.find_all('td'):
                        if td['data-stat'] == "college_link":
                            if td.find_all('a'):
                                for a in td.find_all('a'):
                                    ncaa_links.append(a['href'])
                            else:
                                ncaa_links.append('N/A')
                        elif td['data-stat'] == "player":
                            try:
                                player_ids.append(td['data-append-csv'])
                            except KeyError:
                                player_ids.append('N/A')
                            
            # insert year, player id, ncaa link to table
            df.insert(0, "NCAA_Link", ncaa_links)
            df.insert(0, "Player_ID", player_ids)
            df.insert(0, "Year", year)
            
            # append df to dfs list
            dfs.append(df)

        except Exception as e:
            # store the url and the error it causes in a list
            error = [url, e] 
            # append it to the list of errors
            error_list.append(error)
    
    # print errors
    print(error_list)
    # concatenate dfs into a single df
    draft_df = pd.concat(dfs, ignore_index=True)
    draft_df.to_csv('nfl_draft_2000_2020.csv', index=False)

In [3]:
pullDraftData(years_nfl)

[]


#### 3.3 Pull NCAA team stats

In [13]:
# get ncaa team offense data
def pullTeamOffData(years):    
    # a list to store dataframes -- 1 df per year
    dfs = []
    # a list to store any errors that may come up while scraping
    error_list = []
    
    # iterate over years
    for year in years:
        # use try/except block to catch and inspect any urls that cause an error
        try:
            # set url
            url = 'https://www.sports-reference.com/cfb/years/' + year + '-team-offense.html'
            
            # put random delays between get requests
            delay = np.random.choice(delays)
            time.sleep(delay)
            # make get request with header
            html = requests.get(url, headers=headers)

            # create the BeautifulSoup object
            soup = BeautifulSoup(html.content, "lxml")
            
            # get team offense table
            table = soup.find('table', {'id': 'offense'})
            
            # read table as dataframe
            df = pd.read_html(str(table))[0]
            # remove duplicate header rows
            df.drop_duplicates(keep=False, inplace=True)
            # remove multilevel headers (first header row)
            df.columns = df.columns.droplevel(0)
            
            # add year
            df.insert(0, "Year", year)
            
            # append df to dfs list
            dfs.append(df)
        
        except Exception as e:
            # store the url and the error it causes in a list
            error = [url, e] 
            # append it to the list of errors
            error_list.append(error)
    
    # print errors
    print(error_list)
    # concatenate dataframes into a single dataframe
    team_offense_df = pd.concat(dfs, ignore_index=True)
    # save dataframe to csv
    team_offense_df.to_csv('ncaaf_team_offense_1999_2019.csv', index=False)


In [14]:
pullTeamOffData(years_ncaa)

[['https://www.sports-reference.com/cfb/years/1999-team-offense.html', ValueError('No tables found')]]


In [9]:
# get ncaa team defense data
def pullTeamDefData(years):    
    # a list to store dataframes -- 1 df per year
    dfs = []
    # a list to store any errors that may come up while scraping
    error_list = []
    
    # iterate over years
    for year in years:
        # use try/except block to catch and inspect any urls that cause an error
        try:
            # set url
            url = 'https://www.sports-reference.com/cfb/years/' + year + '-team-defense.html'
            # put random delays between get requests
            delay = np.random.choice(delays)
            time.sleep(delay)
            # get html
            html = requests.get(url, headers=headers)

            # create the BeautifulSoup object
            soup = BeautifulSoup(html.content, "lxml")
            
            # get team defense table
            table = soup.find('table', {'id': 'defense'})
            df = pd.read_html(str(table))[0]
            # remove duplicate header rows
            df.drop_duplicates(keep=False, inplace=True)
            # remove multilevel headers
            df.columns = df.columns.droplevel(0)
            
            # add year
            df.insert(0, "Year", year)
            
            # append dataframe to dataframes list
            dfs.append(df)
        
        except Exception as e:
            # store the url and the error it causes in a list
            error = [url, e] 
            # append it to the list of errors
            error_list.append(error)
    
    # print errors
    print(error_list)
    # concatenate dataframes into a single dataframe
    team_defense_df = pd.concat(dfs, ignore_index=True)
    # save dataframe to csv
    team_defense_df.to_csv('ncaaf_team_defense_1999_2019.csv', index=False)


In [10]:
pullTeamDefData(years_ncaa)

[['https://www.sports-reference.com/cfb/years/1999-team-defense.html', ValueError('No tables found')]]


In [52]:
# get NCAAF ratings
def pullTeamRatings(years):    
    # a list to store dataframes -- 1 df per year
    dfs = []
    # a list to store any errors that may come up while scraping
    error_list = []
    
    # iterate over years
    for year in years:
        # use try/except block to catch and inspect any urls that cause an error
        try:
            # set url
            url = 'https://www.sports-reference.com/cfb/years/' + year + '-ratings.html'
            
            # put random delays between get requests
            delay = np.random.choice(delays)
            time.sleep(delay)
            # make get request with header
            html = requests.get(url, headers=headers)

            # create the BeautifulSoup object
            soup = BeautifulSoup(html.content, "lxml")
            
            # get ratings table
            table = soup.find('table', {'id': 'ratings'})
            
            # read table as dataframe
            df = pd.read_html(str(table))[0]
            # remove duplicate header rows
            df.drop_duplicates(keep=False, inplace=True)
            # remove multilevel headers (first header row)
            df.columns = df.columns.droplevel(0)
            
            # add year
            df.insert(0, "Year", year)
            
            # append df to dfs list
            dfs.append(df)
        
        except Exception as e:
            # store the url and the error it causes in a list
            error = [url, e] 
            # append it to the list of errors
            error_list.append(error)
    
    # print errors
    print(error_list)
    # concatenate dataframes into a single dataframe
    team_ratings_df = pd.concat(dfs, ignore_index=True)
    # save dataframe to csv
    team_ratings_df.to_csv('ncaaf_team_ratings_1999_2019.csv', index=False)


In [53]:
pullTeamRatings(years_ncaa)

[]


#### 3.4 Pull NFL team stats

In [4]:
# get nfl team stats
def pullNFLTeamStats(years):    
    # a list to store dataframes -- 1 df per year
    dfs = []
    # a list to store any errors that may come up while scraping
    error_list = []
    
    # iterate over years
    for year in years:
        # use try/except block to catch and inspect any urls that cause an error
        try:
            # set url
            url = 'https://www.pro-football-reference.com/years/' + year + '/'
            # get html
            delay = np.random.choice(delays)
            time.sleep(delay)
            html = requests.get(url, headers=headers)

            # create the BeautifulSoup object
            soup = BeautifulSoup(html.content, "lxml")
            
            # get NFL team stats
            table = soup.find_all('table', {'id': ['AFC', 'NFC']})
            # read table as dataframe
            tables = pd.read_html(str(table))
            
            # concatenate tables into single dataframe
            df = pd.concat(tables, ignore_index=True)
            
            # remove division rows
            df = df[~df['Tm'].isin(divisions)]
            
            # add year
            df.insert(0, "Year", year)
            
            # append dataframe to dataframes list
            dfs.append(df)
        
        except Exception as e:
            # store the url and the error it causes in a list
            error = [url, e] 
            # append it to the list of errors
            error_list.append(error)
    
    # print errors
    print(error_list)
    # concatenate dataframes into a single dataframe
    nfl_team_stats_df = pd.concat(dfs, ignore_index=True)
    
    # save dataframe to csv
    nfl_team_stats_df.to_csv('nfl_team_stats_1999_2019.csv', index=False)

In [5]:
pullNFLTeamStats(years_ncaa)

C:\Users\Roland\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



#### 3.5 Pull player stats (minimum 14 Att/G, 75% of school games played)

In [ ]:
# get ncaa player data
def pullPlayerData(years, tableID):
    # a list to store dataframes -- 1 df per year
    dfs = []
    # a list to store any errors that may come up while scraping
    error_list = []
    # list of tables with multilevel headers
    multi_list = ['passing','rushing','receiving']
    
    # iterate over years
    for year in years:
        # use try/except block to catch and inspect any urls that cause an error
        try:
            # set url
            url = 'https://www.sports-reference.com/cfb/years/' + year + '-' + tableID + '.html'
            
            # put random delays between get requests
            delay = np.random.choice(delays)
            time.sleep(delay)
            # make get request with header
            html = requests.get(url, headers=headers)

            # create the BeautifulSoup object
            soup = BeautifulSoup(html.content, "lxml")
            # get table
            table = soup.find('table', {'id': tableID})
            
            # read table as dataframe
            df = pd.read_html(str(table))[0]
            # remove duplicate header rows
            df.drop_duplicates(keep=False, inplace=True)
            # remove multiheaders (first header row)
            if tableID in multi_list:
                df.columns = df.columns.droplevel(0)

            player_ncaa_ids = []
            ncaa_links = []
            for tbody in table.find_all('tbody'):
                for tr in tbody.find_all('tr'):
                    for td in tr.find_all('td'):
                        if td['data-stat'] == "player":
                            try:
                                player_ncaa_ids.append(td['data-append-csv'])
                            except KeyError:
                                player_ncaa_ids.append('N/A')
                            
                            if td.find_all('a'):
                                for a in td.find_all('a'):
                                    ncaa_links.append(a['href'])
                            else:
                                ncaa_links.append('N/A')
                            
            # insert year, player id, ncaa link to table
            df.insert(0, "NCAA_Link", ncaa_links)
            df.insert(0, "Player_NCAA_ID", player_ncaa_ids)
            df.insert(0, "Year", year)
            
            # append df to dfs list
            dfs.append(df)

        except Exception as e:
            # store the url and the error it causes in a list
            error = [url, e] 
            # append it to the list of errors
            error_list.append(error)
    
    # print errors
    print(error_list)
    # concatenate dataframes into a single dataframe
    out_df = pd.concat(dfs, ignore_index=True)
    out_df.to_csv('ncaa_player_' + tableID + '_stats_1999_2019.csv', index=False)

In [ ]:
for table in player_tables:
    pullPlayerData(years, table)

### 3.6 Pull Consensus All-Americans

In [159]:
# pull All-Americans
def pullAllAmericans(years):

    base_url = 'https://www.sports-reference.com'
    lst = []
    
    # a list to store any errors that may come up while scraping
    error_list = []
    
    # iterate over years
    for year in years:
        # use try/except block to catch and inspect any urls that cause an error
        try:
            # set url
            url = 'https://www.sports-reference.com/cfb/years/' + year + '.html'
            
            # put random delays between get requests 
            delay = np.random.choice(delays)
            time.sleep(delay)
            
            # get html
            res = requests.get(url, headers = headers)
            
            # Work around comments
            comm = re.compile("<!--|-->")
            soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
            for row in soup.find_all('div', id = 'div_all_americans'):
                for p in row.find_all('p'):
                    for a in p.find_all('a',limit=1):
                        line = [year, p.text.split(',')[0].replace('*',''), p.text.split(',')[1].replace(' ',''), p.text.split(',')[2].lstrip(), base_url + a['href']]
                    
                    lst.append(line)
        
        except Exception as e:
            # store the url and the error it causes in a list
            error = [url, e] 
            # append it to the list of errors
            error_list.append(error)
    
    # print errors
    print(error_list)
    # add list to dataframe
    df=pd.DataFrame(lst,columns=['Year','Player','Pos','School','NCAA_Link'])
    # save as csv
    df.to_csv('ncaa_all_americans__1999_2019.csv', index=False)
    

In [160]:
pullAllAmericans(years_ncaa)

[]


#### Pull player stats (minimum 14 Att/G, 75% of school games played) -- OBSOLETE

In [12]:
'''
# get ncaa player passing data
def pullPassingData(years):    
    # a list to store dataframes -- 1 df per year
    dfs = []
    # a list to store any errors that may come up while scraping
    error_list = []
    
    # iterate over years
    for year in years:
        # use try/except block to catch and inspect any urls that cause an error
        try:
            # set url
            url = 'https://www.sports-reference.com/cfb/years/' + year + '-passing.html'
            
            # put random delays between get requests
            delay = np.random.choice(delays)
            time.sleep(delay)
            # make get request with header
            html = requests.get(url, headers=headers)

            # create the BeautifulSoup object
            soup = BeautifulSoup(html.content, "lxml")
            # get passing table
            table = soup.find('table', {'id': 'passing'})
            
            # read table as dataframe
            df = pd.read_html(str(table))[0]
            # remove duplicate header rows
            df.drop_duplicates(keep=False, inplace=True)
            # remove multiheaders (first header row)
            df.columns = df.columns.droplevel(0)

            player_ncaa_ids = []
            ncaa_links = []
            for tbody in table.find_all('tbody'):
                for tr in tbody.find_all('tr'):
                    for td in tr.find_all('td'):
                        if td['data-stat'] == "player":
                            try:
                                player_ncaa_ids.append(td['data-append-csv'])
                            except KeyError:
                                player_ncaa_ids.append('N/A')
                            
                            if td.find_all('a'):
                                for a in td.find_all('a'):
                                    ncaa_links.append(a['href'])
                            else:
                                ncaa_links.append('N/A')
                            
            # insert year, player id, ncaa link to table
            df.insert(0, "NCAA_Link", ncaa_links)
            df.insert(0, "Player_NCAA_ID", player_ncaa_ids)
            df.insert(0, "Year", year)
            
            # append df to dfs list
            dfs.append(df)

        except Exception as e:
            # store the url and the error it causes in a list
            error = [url, e] 
            # append it to the list of errors
            error_list.append(error)
    
    # print errors
    print(error_list)
    #print(len(ncaa_links))
    #print(len(player_ncaa_ids))
    #print(df.head())
    passing_df = pd.concat(dfs, ignore_index=True)
    passing_df.to_csv('ncaa_player_passing_stats_1999_2019.csv', index=False)
'''

In [13]:
#pullPassingData(years_ncaa)

[]


In [15]:
'''
# get ncaa player rushing data
def pullRushingData(years):    
    # a list to store dataframes -- 1 df per year
    dfs = []
    # a list to store any errors that may come up while scraping
    error_list = []
    
    # iterate over years
    for year in years:
        # use try/except block to catch and inspect any urls that cause an error
        try:
            # set url
            url = 'https://www.sports-reference.com/cfb/years/' + year + '-rushing.html'
            
            # put random delays between get requests
            delay = np.random.choice(delays)
            time.sleep(delay)
            # make get request with header
            html = requests.get(url, headers=headers)

            # create the BeautifulSoup object
            soup = BeautifulSoup(html.content, "lxml")
            # get rushing table
            table = soup.find('table', {'id': 'rushing'})
            
            # read table as dataframe
            df = pd.read_html(str(table))[0]
            # remove duplicate header rows
            df.drop_duplicates(keep=False, inplace=True)
            # remove multiheaders (first header row)
            df.columns = df.columns.droplevel(0)

            player_ncaa_ids = []
            ncaa_links = []
            for tbody in table.find_all('tbody'):
                for tr in tbody.find_all('tr'):
                    for td in tr.find_all('td'):
                        if td['data-stat'] == "player":
                            try:
                                player_ncaa_ids.append(td['data-append-csv'])
                            except KeyError:
                                player_ncaa_ids.append('N/A')
                            
                            if td.find_all('a'):
                                for a in td.find_all('a'):
                                    ncaa_links.append(a['href'])
                            else:
                                ncaa_links.append('N/A')
                            
            # insert year, player id, ncaa link to table
            df.insert(0, "NCAA_Link", ncaa_links)
            df.insert(0, "Player_NCAA_ID", player_ncaa_ids)
            df.insert(0, "Year", year)
            
            # append df to dfs list
            dfs.append(df)

        except Exception as e:
            # store the url and the error it causes in a list
            error = [url, e] 
            # append it to the list of errors
            error_list.append(error)
    
    # print errors
    print(error_list)
    #print(len(ncaa_links))
    #print(len(player_ncaa_ids))
    #print(df.head())
    rushing_df = pd.concat(dfs, ignore_index=True)
    rushing_df.to_csv('ncaa_player_rushing_stats_1999_2019.csv', index=False)
'''

In [16]:
#pullRushingData(years_ncaa)

[]


In [19]:
'''
# get ncaa player receiving data
def pullReceivingData(years):    
    # a list to store dataframes -- 1 df per year
    dfs = []
    # a list to store any errors that may come up while scraping
    error_list = []
    
    # iterate over years
    for year in years:
        # use try/except block to catch and inspect any urls that cause an error
        try:
            # set url
            url = 'https://www.sports-reference.com/cfb/years/' + year + '-receiving.html'
            
            # put random delays between get requests
            delay = np.random.choice(delays)
            time.sleep(delay)
            # make get request with header
            html = requests.get(url, headers=headers)

            # create the BeautifulSoup object
            soup = BeautifulSoup(html.content, "lxml")
            # get receiving table
            table = soup.find('table', {'id': 'receiving'})
            
            # read table as dataframe
            df = pd.read_html(str(table))[0]
            # remove duplicate header rows
            df.drop_duplicates(keep=False, inplace=True)
            # remove multiheaders (first header row)
            df.columns = df.columns.droplevel(0)

            player_ncaa_ids = []
            ncaa_links = []
            for tbody in table.find_all('tbody'):
                for tr in tbody.find_all('tr'):
                    for td in tr.find_all('td'):
                        if td['data-stat'] == "player":
                            try:
                                player_ncaa_ids.append(td['data-append-csv'])
                            except KeyError:
                                player_ncaa_ids.append('N/A')
                            
                            if td.find_all('a'):
                                for a in td.find_all('a'):
                                    ncaa_links.append(a['href'])
                            else:
                                ncaa_links.append('N/A')
                            
            # insert year, player id, ncaa link to table
            df.insert(0, "NCAA_Link", ncaa_links)
            df.insert(0, "Player_NCAA_ID", player_ncaa_ids)
            df.insert(0, "Year", year)
            
            # append df to dfs list
            dfs.append(df)

        except Exception as e:
            # store the url and the error it causes in a list
            error = [url, e] 
            # append it to the list of errors
            error_list.append(error)
    
    # print errors
    print(error_list)
    #print(len(ncaa_links))
    #print(len(player_ncaa_ids))
    #print(df.head())
    receiving_df = pd.concat(dfs, ignore_index=True)
    receiving_df.to_csv('ncaa_player_receiving_stats_1999_2019.csv', index=False)
'''

In [20]:
#pullReceivingData(years_ncaa)

[]


In [23]:
'''
# get ncaa player kicking data
def pullKickingData(years):    
    # a list to store dataframes -- 1 df per year
    dfs = []
    # a list to store any errors that may come up while scraping
    error_list = []
    
    # iterate over years
    for year in years:
        # use try/except block to catch and inspect any urls that cause an error
        try:
            # set url
            url = 'https://www.sports-reference.com/cfb/years/' + year + '-kicking.html'
            
            # put random delays between get requests
            delay = np.random.choice(delays)
            time.sleep(delay)
            # make get request with header
            html = requests.get(url, headers=headers)

            # create the BeautifulSoup object
            soup = BeautifulSoup(html.content, "lxml")
            # get kicking table
            table = soup.find('table', {'id': 'kicking'})
            
            # read table as dataframe
            df = pd.read_html(str(table))[0]
            # remove duplicate header rows
            df.drop_duplicates(keep=False, inplace=True)

            player_ncaa_ids = []
            ncaa_links = []
            for tbody in table.find_all('tbody'):
                for tr in tbody.find_all('tr'):
                    for td in tr.find_all('td'):
                        if td['data-stat'] == "player":
                            try:
                                player_ncaa_ids.append(td['data-append-csv'])
                            except KeyError:
                                player_ncaa_ids.append('N/A')
                            
                            if td.find_all('a'):
                                for a in td.find_all('a'):
                                    ncaa_links.append(a['href'])
                            else:
                                ncaa_links.append('N/A')
                            
            # insert year, player id, ncaa link to table
            df.insert(0, "NCAA_Link", ncaa_links)
            df.insert(0, "Player_NCAA_ID", player_ncaa_ids)
            df.insert(0, "Year", year)
            
            # append df to dfs list
            dfs.append(df)

        except Exception as e:
            # store the url and the error it causes in a list
            error = [url, e] 
            # append it to the list of errors
            error_list.append(error)
    
    # print errors
    print(error_list)
    #print(len(ncaa_links))
    #print(len(player_ncaa_ids))
    #print(df.head())
    kicking_df = pd.concat(dfs, ignore_index=True)
    kicking_df.to_csv('ncaa_player_kicking_stats_1999_2019.csv', index=False)
'''

In [24]:
#pullKickingData(years_ncaa)

[['https://www.sports-reference.com/cfb/years/1999-kicking.html', ValueError('No tables found')]]


In [25]:
'''
# get ncaa player punting data
def pullPuntingData(years):    
    # a list to store dataframes -- 1 df per year
    dfs = []
    # a list to store any errors that may come up while scraping
    error_list = []
    
    # iterate over years
    for year in years:
        # use try/except block to catch and inspect any urls that cause an error
        try:
            # set url
            url = 'https://www.sports-reference.com/cfb/years/' + year + '-punting.html'
            
            # put random delays between get requests
            delay = np.random.choice(delays)
            time.sleep(delay)
            # make get request with header
            html = requests.get(url, headers=headers)

            # create the BeautifulSoup object
            soup = BeautifulSoup(html.content, "lxml")
            # get punting table
            table = soup.find('table', {'id': 'punting'})
            
            # read table as dataframe
            df = pd.read_html(str(table))[0]
            # remove duplicate header rows
            df.drop_duplicates(keep=False, inplace=True)

            player_ncaa_ids = []
            ncaa_links = []
            for tbody in table.find_all('tbody'):
                for tr in tbody.find_all('tr'):
                    for td in tr.find_all('td'):
                        if td['data-stat'] == "player":
                            try:
                                player_ncaa_ids.append(td['data-append-csv'])
                            except KeyError:
                                player_ncaa_ids.append('N/A')
                            
                            if td.find_all('a'):
                                for a in td.find_all('a'):
                                    ncaa_links.append(a['href'])
                            else:
                                ncaa_links.append('N/A')
                            
            # insert year, player id, ncaa link to table
            df.insert(0, "NCAA_Link", ncaa_links)
            df.insert(0, "Player_NCAA_ID", player_ncaa_ids)
            df.insert(0, "Year", year)
            
            # append df to dfs list
            dfs.append(df)

        except Exception as e:
            # store the url and the error it causes in a list
            error = [url, e] 
            # append it to the list of errors
            error_list.append(error)
    
    # print errors
    print(error_list)
    #print(len(ncaa_links))
    #print(len(player_ncaa_ids))
    #print(df.head())
    punting_df = pd.concat(dfs, ignore_index=True)
    punting_df.to_csv('ncaa_player_punting_stats_1999_2019.csv', index=False)
'''

In [26]:
#pullPuntingData(years_ncaa)

[['https://www.sports-reference.com/cfb/years/1999-punting.html', ValueError('No tables found')]]
